<p style="padding: 10px; border: 1px solid black;">
<img src="../mlu_utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>


# <a name="0">Machine Learning University</a>
## <a name="0">Lab2a: Amazon Bedrock API Walkthrough</a>

<br>

---
    
Amazon Bedrock is a fully managed service that makes foundation models (FMs) from Amazon and third-party model providers easily accessible through an API. This notebook covers the Amazon Bedrock API using SDK for Python (Boto3).
    
1. <a href="#1">Installation and API calls</a>
1. <a href="#2">Accessing the Bedrock service using the SDK for Python</a>
2. <a href="#3">Bedrock API Call</a>
3. <a href="#4">Quizzes</a>

In this lab, we will cover topics such as:
- Setting up and accessing the Bedrock service using Boto3
- Exploring available Large Language Models (LLMs) in Bedrock
- Performing Bedrock API calls with various customization options
- Understanding and manipulating model parameters for text generation

### <a name="1">Installation of libraries </a>

We first install recent versions of boto3 and botocore (defined in a file called `requirements.txt`).

In [ ]:
%%capture
!pip install -q -r ../requirements.txt

We can access the Bedrock service through boto3 by providing the service name, region name and endpoint URL.

### <a name="2">Accessing the Bedrock service using the SDK for Python</a>

In [ ]:
import json, boto3

session = boto3.session.Session()

bedrock = session.client(
    service_name="bedrock",
    region_name=session.region_name,
)

Let's take a look at the available Large Language Models (LLMs) in Bedrock.

In [ ]:
bedrock.list_foundation_models()["modelSummaries"]

We use the service name 'bedrock-runtime' for inference. 

In [ ]:
# For inference
bedrock_inference = session.client(
    service_name="bedrock-runtime",
)

Model IDs can be used to select a specific model in the API calls. 

This demo is focusing on the __Amazon Titan Text G1 - Premier__ model from __Amazon__. We will use the model with id: `amazon.titan-text-premier-v1:0`

---

### <a name="3">Bedrock API Call</a>

Bedrock API has the following parameters:
* __`body`:__ The message body that includes the input text and model parameters. Model parameters help customize the models and the generated outputs.
* __`modelId`:__ Identifier of the model. We can pick a model from the list of models printed in the previous code block.
* __`accept`:__ The desired type of the inference body in the response.
* __`contentType`:__ The type of the input data in the request body.

<br>

__Model parameters:__

These parameters are provided within the body of the API call. Basically, we can control the randomness and the length of the generated sequences. 

__Randomness:__ 

* __`temperature`:__ Controls the randomness of the generated sequences. This parameter is between zero and one. When set closer to zero, the model tends to select higher probability words. When set further away from zero, the model may select lower-probability words. Temperature of zero gives the same output for the same input at every run.
* __`topP`:__ Top P defines a cut-off based on the sum of probabilities of the potential choices. With this cut-off, the model only selects from the most probable words whose probabilities sum up to the topP value. 

__Length:__ 

* __`maxTokens`:__ Controls the maximum number of tokens in the generated response.
* __`stopSequences`:__ A sequence of characters to stop the model from generating its output.

Let's define a function that will build our prompt as well as pass some model parameters to Bedrock. Once the call parameters are ready, we can use the __invoke_model()__ function to send the data and collect the response from the model. Then, we return the response at the end.

**Note:** Different models may have different model parameters. Please refer to the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/service_code_examples_bedrock-runtime.html) for more details.

In [ ]:
def send_prompt(prompt_data, temperature=0.0, top_p=1.0, max_token_count=1000):

    body = json.dumps(
        {
            "inputText": prompt_data,
            "textGenerationConfig": {
                "temperature": temperature,
                "topP": top_p,
                "maxTokenCount": max_token_count,
            },
        }
    )

    modelId = "amazon.titan-text-premier-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_inference.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )

    response_body = json.loads(response["body"].read())

    return response_body["results"][0]["outputText"]

Let's construct a simple API call and run it. 

As a sample text input, we use the following: __"Can you name a few real-life applications of natural language processing?"__. 

For inference parameters, we set the __`temperature`__ to 0.

In [ ]:
from IPython.display import Markdown, display

prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(Markdown(send_prompt(prompt_data, temperature=0.0)))

This provides a list of real-life NLP applications. 

If we want to generate slightly different looking outputs, we can increase the __`temperature`__ value. Let's also set the __`maxTokens`__ parameter this time.

Changing the temperature parameter will create a slightly different looking list.

In [ ]:
prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(Markdown(send_prompt(prompt_data, temperature=0.5, max_token_count=450)))

In addition to the __`temperature`__ and __`maxTokens`__ parameters, we can also add in the __`topP`__ parameter to set a cut-off for the sum of the probabilities of the potential words.

In [ ]:
prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(
    Markdown(send_prompt(prompt_data, max_token_count=450, temperature=0.5, top_p=0.7))
)

---
### <a name="4">Quiz Questions</a>

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="../mlu_utils/challenge.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Answer the following questions to test your understanding of using LLMs for inference.</p>
    <br>
</div>

In [ ]:
import sys
sys.path.append('..')
from mlu_utils.quiz_questions import *

lab2a_question1

# Thank you!

<p style="padding: 10px; border: 1px solid black;">
<img src="../mlu_utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>